In [1]:
ANIMAL = "chimp"

In [2]:
# Parameters
ANIMAL = "hamster"


In [3]:
import pathlib
import keras
import tensorflow as tf
import plotly.graph_objects as go
import contextlib
import io

from petface.const import IMG_HEIGHT, IMG_WIDTH, ROOT_DIR
from petface.face_identification.model import get_reidentification_model

with contextlib.redirect_stdout(io.StringIO()):
    IMG_DIMENSIONS = (IMG_HEIGHT, IMG_WIDTH)

    root_dir = pathlib.Path(ROOT_DIR)
    test_dir = root_dir / "dataset" / "out" / "images" / "test" / ANIMAL

    test_ds = keras.utils.image_dataset_from_directory(
        directory=test_dir,
        image_size=IMG_DIMENSIONS,
        batch_size=32
    )

    eval_ds = test_ds.prefetch(tf.data.AUTOTUNE)  # type: ignore

2025-12-01 19:54:44.223781: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-01 19:54:44.248324: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2025-12-01 19:54:44.737953: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


I0000 00:00:1764636885.225232  309766 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6127 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.9


In [4]:
with contextlib.redirect_stdout(io.StringIO()):
    softmax_cross_entropy_model = get_reidentification_model(
        animal=ANIMAL, loss_type="cross_entropy", classification_type="softmax"
    )
    softmax_similarity_model = get_reidentification_model(
        animal=ANIMAL, loss_type="cross_entropy", classification_type="cosine_similarity"
    )
    arcface_cross_entropy_model = get_reidentification_model(
        animal=ANIMAL, loss_type="arcface", classification_type="softmax"
    )
    arcface_similarity_model = get_reidentification_model(
        animal=ANIMAL, loss_type="arcface", classification_type="cosine_similarity"
    )

    softmax_cross_entropy_model.load_from_checkpoint()
    softmax_similarity_model.load_from_checkpoint()
    arcface_cross_entropy_model.load_from_checkpoint()
    arcface_similarity_model.load_from_checkpoint()

/home/mtofano/micromamba/envs/petface/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 84 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


/home/mtofano/micromamba/envs/petface/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 80 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


/home/mtofano/micromamba/envs/petface/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 78 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
softmax_cross_entropy_model.display_training_history()

In [6]:
softmax_similarity_model.display_training_history()

In [7]:
arcface_cross_entropy_model.display_training_history()

In [8]:
arcface_similarity_model.display_training_history()

In [9]:
with contextlib.redirect_stdout(io.StringIO()):
    softmax_cross_entropy_eval_results = softmax_cross_entropy_model.evaluate(eval_ds)
    softmax_similarity_eval_results = softmax_similarity_model.evaluate(eval_ds)
    arcface_cross_entropy_eval_results = arcface_cross_entropy_model.evaluate(eval_ds)
    arcface_similarity_eval_results = arcface_similarity_model.evaluate(eval_ds)

2025-12-01 19:54:51.031709: I external/local_xla/xla/service/service.cc:163] XLA service 0x7e4434002bf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-01 19:54:51.031737: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Ti, Compute Capability 8.9
2025-12-01 19:54:51.120233: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


2025-12-01 19:54:51.377482: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 90300


2025-12-01 19:54:51.595470: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1725', 20 bytes spill stores, 20 bytes spill loads



I0000 00:00:1764636893.802664  309912 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2025-12-01 19:54:56.892938: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


2025-12-01 19:54:57.316873: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


2025-12-01 19:55:11.944616: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [10]:
eval_table = go.Figure(
    data=[
        go.Table(
            header={"values": ["Loss", "Classifier", "Top 1 Accuracy", "Top 5 Accuracy", "Top 50 Accuracy"]},
            cells={
                "values": [
                    ["Cross Entropy", "Cross Entropy", "Arcface", "Arcface"],
                    ["Softmax", "Cosine Similarity", "Softmax", "Cosine Similarity"],
                    [
                        softmax_cross_entropy_eval_results["top_1_accuracy"],
                        softmax_similarity_eval_results["top_1_accuracy"],
                        arcface_cross_entropy_eval_results["top_1_accuracy"],
                        arcface_similarity_eval_results["top_1_accuracy"]
                    ],
                    [
                        softmax_cross_entropy_eval_results["top_5_accuracy"],
                        softmax_similarity_eval_results["top_5_accuracy"],
                        arcface_cross_entropy_eval_results["top_5_accuracy"],
                        arcface_similarity_eval_results["top_5_accuracy"]
                    ],
                    [
                        softmax_cross_entropy_eval_results["top_50_accuracy"],
                        softmax_similarity_eval_results["top_50_accuracy"],
                        arcface_cross_entropy_eval_results["top_50_accuracy"],
                        arcface_similarity_eval_results["top_50_accuracy"]
                    ],
                ]
            }
        )
    ]
)

eval_table.show()